In [ ]:
import os
import pdfplumber
import pandas as pd
import numpy as np
import re

# libro = pd.read_excel('file_informeMensualREC - 2025-12-02T162142.258.xlsx', skiprows=5)
# libro

In [ ]:
ruta_carpeta = 'boletas_pdf/'
df_glosa = pd.DataFrame(columns=['Numero Boleta', 'RUT', 'Glosa', 'Valor'])

for file in os.listdir(ruta_carpeta):
    ruta_pdf = ruta_carpeta + file

    with pdfplumber.open(ruta_pdf) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()
        lines = text.split('\n')


    id_boleta = 'N ° '
    for linea in lines:
        if id_boleta in linea:
            id_boleta_valor = linea.split(id_boleta)[1].strip()
            print(f'ID Boleta: {id_boleta_valor}')

    id_rut = 'RUT: '
    for linea in lines:
        if id_rut in linea:
            id_rut_valor = linea.split(id_rut)[1].strip()
            print(f'RUT: {id_rut_valor}')


    ini = 'Por atención profesional:'
    fin = 'Total Honorarios'
    flag = False

    for linea in lines:
        if ini in linea:
            flag = True
            continue
        
        if fin in linea:
            flag = False
            continue
        
        if flag == True:
            glosa = linea.strip()
            
            valor_ = glosa.split()[-1]
            if '.' in valor_:
                valor = np.int64(valor_.replace('.', ''))
                glosa = glosa.replace(valor_, '')
            else:
                valor = None

            df_glosa = pd.concat([df_glosa, pd.DataFrame({'Numero Boleta': [id_boleta_valor], 'RUT': [id_rut_valor], 'Glosa': [glosa], 'Valor':[valor]})], ignore_index=True)

    df_glosa = df_glosa.reset_index(drop=True)
    for ix in df_glosa.index:
        if df_glosa.at[ix, 'Valor'] == None:
            df_glosa.at[ix-1, 'Glosa'] = df_glosa.at[ix-1, 'Glosa'] + ' ' + df_glosa.at[ix, 'Glosa']
            df_glosa = df_glosa.drop(ix)

df_glosa.to_excel('glosas_boleta.xlsx', index=False)
